In [1]:
import pandas as pd
from math import sqrt

In [2]:
df = pd.read_csv("../data/collects.csv")

In [3]:
# Train
unique_sensors = []
for s in df["sensorId"].tolist():
    if s not in unique_sensors:
        unique_sensors.append(s)

In [4]:
print(unique_sensors)

['IAJ9206', 'LZY4270', 'MUR8453', 'MXK6435', 'MYD8706', 'MYS2071', 'MZU6388', 'NAH4736', 'NAI1549', 'NEW4797']


In [5]:
sensors_means = dict()
sensors_std = dict()
for s in unique_sensors:
    aux_df = df[df["sensorId"] == s] # Returns a df with all columns for each sensor.
    # print(aux_df)
    cols_mean = dict()
    cols_std = dict()
    for col in ["params.velRMS.x", "params.velRMS.y", "params.velRMS.z", "params.accelRMS.x", "params.accelRMS.y", "params.accelRMS.z"]:
        col_values = aux_df[col].tolist() # Returns a list with all the values of a column for each sensor.
        # print(col_values)
        col_sum = 0
        counter = 0
        for v in col_values:
            if not pd.isna(v): # Exclude NaN values.
                col_sum += v
                counter += 1
        cols_mean[col] = col_sum/counter # Returns the mean of column "a" of the sensor "b"
        # print(cols_mean[col])

        col_std = 0
        counter = -1
        for v in col_values:
            if not pd.isna(v): # Exclude NaN values.
                col_std += (v-cols_mean[col])**2
                counter += 1
        
        cols_std[col] = sqrt(col_std/counter) # Returns the std dev of column "a" of the sensor "b"
    # print(cols_mean)
    sensors_means[s] = cols_mean
    sensors_std[s] = cols_std

# print(sensors_means)

# Predict
new_value = 9e-2


in_alert = dict()
for s in unique_sensors:
    cols_in_alert = dict()
    for col in ["params.velRMS.x", "params.velRMS.y", "params.velRMS.z", "params.accelRMS.x", "params.accelRMS.y", "params.accelRMS.z"]:
        if new_value >= sensors_means[s][col] +  2*sensors_std[s][col]:
            cols_in_alert[col] = True
        elif new_value < sensors_means[s][col] + 2*sensors_std[s][col]:
            cols_in_alert[col] = False

    in_alert[s] = cols_in_alert


In [6]:
df = pd.DataFrame.from_dict(in_alert, orient='index')
print(df)

         params.velRMS.x  params.velRMS.y  params.velRMS.z  params.accelRMS.x  \
IAJ9206             True             True             True               True   
LZY4270             True             True             True               True   
MUR8453             True             True             True               True   
MXK6435             True             True             True               True   
MYD8706             True             True             True              False   
MYS2071             True             True             True               True   
MZU6388             True             True             True               True   
NAH4736             True             True             True              False   
NAI1549             True             True             True               True   
NEW4797             True             True             True              False   

         params.accelRMS.y  params.accelRMS.z  
IAJ9206              False               True  
LZY4270     